# Pre-processing and Feature Engineeing

In [1]:
import sys
from pathlib import Path

project_root = Path().resolve().parent

sys.path.append(str(project_root))

In [2]:
import pandas as pd
import numpy as np

from config.features import get_features

In [5]:
train_identity = pd.read_csv('../data/train_identity.csv')
identity_columns = train_identity.columns.difference(['TransactionID'])

print("Identity dataset load sucessfully!")

Identity dataset load sucessfully!


In [4]:
merged_chunks = []
transaction_chunks = pd.read_csv('../data/train_transaction.csv', chunksize=10 ** 5)

for chunk in transaction_chunks:
    new_chunk = chunk.merge(train_identity, on='TransactionID', how='left')
    merged_chunks.append(new_chunk)

train = pd.concat(merged_chunks)
train.shape

(590540, 434)

In [6]:
train['hasIdentity'] = (~train[identity_columns].isna().all(axis=1)).astype(int)

In [7]:
train = train[get_features(train.columns)]
train.head(5)

Existing features: 435
Relevant features: 290
Final training set features: 281


,TransactionID,hasIdentity,isFraud,ProductCD,card1,card2,card3,card4,card5,card6,...,id_20,id_30,id_31,id_32,id_33,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,0,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,1,0,H,4497,514.0,150.0,mastercard,102.0,credit,...,144.0,Android 7.0,samsung browser 6.2,32.0,2220x1080,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


Make sure to delete the unnecessary variables from memory to relieve some of it, so it will not bother us throughout the script. Also, let's save this merged dataset to optimize future reading operations.

In [8]:
del train_identity, transaction_chunks

In [ ]:
train.to_parquet('../data/train.parquet', index=False)